## Solving a system with a `banded Jacobian matrix`

`odeint` can be told that the `Jacobian` is __banded__. For a _large system of differential equations_ that are known to be __stiff__, this can `improve performance significantly`.

As an example, we'll solve the `1-D Gray-Scott partial differential equations` using the _method of lines [`MOL`]_. The `Gray-Scott equations` for the functions $u(x, t)$ and $v(x, t)$ on the interval $x \in [0, L]$ are

$\frac{\partial u}{\partial t} = D_u \frac{\partial^2 u}{\partial x^2} - uv^2 + f(1-u) \\$
$\frac{\partial v}{\partial t} = D_v \frac{\partial^2 v}{\partial x^2} + uv^2 - (f + k)v \\$

where $D_u$ and $D_v$ are the `diffusion coefficients` of the components $u$ and $v$, respectively, and $f$ and $k$ are `constants`. (For more information about the system, see [http://groups.csail.mit.edu/mac/projects/amorphous/GrayScott/](http://groups.csail.mit.edu/mac/projects/amorphous/GrayScott/))

We'll assume `Neumann boundary conditions` (i.e., `"no flux"`):

$\frac{\partial u}{\partial x}(0,t) = 0, \quad$
$\frac{\partial v}{\partial x}(0,t) = 0, \quad$
$\frac{\partial u}{\partial x}(L,t) = 0, \quad$
$\frac{\partial v}{\partial x}(L,t) = 0$

To apply the `method of lines`, we __discretize__ the $x$ variable by defining the `uniformly spaced grid` of $N$ points $\left\{x_0, x_1, \ldots, x_{N-1}\right\}$, with $x_0 = 0$ and $x_{N-1} = L$. We define $u_j(t) \equiv u(x_k, t)$ and $v_j(t) \equiv v(x_k, t)$, and replace the $x$ derivatives with _finite differences_. That is,

$\frac{\partial^2 u}{\partial x^2}(x_j, t) \rightarrow
    \frac{u_{j-1}(t) - 2 u_{j}(t) + u_{j+1}(t)}{(\Delta x)^2}$

We then have a `system` of $2N$ `ordinary differential equations`:

$\frac{du_j}{dt} = \frac{D_u}{(\Delta x)^2} \left(u_{j-1} - 2 u_{j} + u_{j+1}\right)
          -u_jv_j^2 + f(1 - u_j) \\
    \frac{dv_j}{dt} = \frac{D_v}{(\Delta x)^2} \left(v_{j-1} - 2 v_{j} + v_{j+1}\right)
          + u_jv_j^2 - (f + k)v_j$

For convenience, the $(t)$ arguments have been dropped.

To __enforce__ the `boundary conditions`, we introduce `"ghost" points` $x_{-1}$ and $x_N$, and define $u_{-1}(t) \equiv u_1(t)$, $u_N(t) \equiv u_{N-2}(t)$; $v_{-1}(t)$ and $v_N(t)$
are defined analogously.

Then,

$\frac{du_0}{dt} = \frac{D_u}{(\Delta x)^2} \left(2u_{1} - 2 u_{0}\right)
          -u_0v_0^2 + f(1 - u_0) \\
    \frac{dv_0}{dt} = \frac{D_v}{(\Delta x)^2} \left(2v_{1} - 2 v_{0}\right)
          + u_0v_0^2 - (f + k)v_0$

and

$\frac{du_{N-1}}{dt} = \frac{D_u}{(\Delta x)^2} \left(2u_{N-2} - 2 u_{N-1}\right)
          -u_{N-1}v_{N-1}^2 + f(1 - u_{N-1}) \\
    \frac{dv_{N-1}}{dt} = \frac{D_v}{(\Delta x)^2} \left(2v_{N-2} - 2 v_{N-1}\right)
          + u_{N-1}v_{N-1}^2 - (f + k)v_{N-1}$

We can now starting implementing this system in code. We must combine $\{u_k\}$ and $\{v_k\}$ into a single vector of length $2N$. The two obvious choices are $\{u_0, u_1, \ldots, u_{N-1}$, $v_0, v_1, \ldots, v_{N-1}\}$ and $\{u_0, v_0, u_1, v_1, \ldots, u_{N-1}$, $v_{N-1}\}$. Mathematically, it does not matter, but the choice affects how efficiently `odeint` can solve the system. The reason is in how the `order` affects the _pattern of the nonzero elements_ of the `Jacobian matrix`.

When the variables are __interleaved__, the _`bandwidth` is much smaller_. The `main diagonal` and the `two diagonals immediately above` and the `two immediately below` the main diagonal are the __nonzero diagonals__. This is important, because the inputs `mu` and `ml` of `odeint` are the _upper and lower bandwidths_ of the `Jacobian matrix`. When the variables are __interleaved__, `mu` and `ml` are 2. When the variables are stacked with $\{v_k\}$ following $\{u_k\}$, the _upper and lower bandwidths_ are N.

With that decision made, we can write the function that implements the system of differential equations.

First, we define the functions for the `source` and `reaction` terms of the system.

In [1]:
from scipy import integrate
from time import time
import numpy as np

In [2]:
def G(u, v, f, k):
    return f * (1 - u) - u * v ** 2


def H(u, v, f, k):
    return -(f + k) * v + u * v ** 2

Next, we define the function that computes the right-hand side of the system of differential equations.

In [3]:
def grayscott1d(y, t, f, k, du, dv, dx):
    """
    Differential equations for the 1-D Gray-Scott equations.
    
    The ODEs are derived using the method of lines.
    """
    # the vectors u and v are interleaved in y
    # we define views of u and v by slicing y
    u = y[::2]
    v = y[1::2]
    
    # dydt is the return value of this function
    dydt = np.empty_like(y)
    
    # just like u and v are views of the interleaved vectors in y
    # dudt and dvdt are views of the interleaved output vectors in dydt
    dudt = dydt[::2]
    dvdt = dydt[1::2]
    
    # compute du/dt and dv/dt
    # the end points and the interior points are handled separately
    dudt[0] = G(u[0], v[0], f, k) + du * (-2.0 * u[0] + 2.0 * u[1]) / dx ** 2
    dudt[1:-1] = G(u[1:-1], v[1:-1], f, k) + du * np.diff(u, 2) / dx ** 2
    dudt[-1] = G(u[-1], v[-1], f, k) + du * (-2.0 * u[-1] + 2.0 * u[-2]) / dx ** 2
    
    dvdt[0] = H(u[0], v[0], f, k) + dv * (-2.0 * v[0] + 2.0 * v[1]) / dx ** 2
    dvdt[1:-1] = H(u[1:-1], v[1:-1], f, k) + dv * np.diff(v, 2) / dx ** 2
    dvdt[-1] = H(u[-1], v[-1], f, k) + dv * (-2.0 * v[-1] + 2.0 * v[-2]) / dx ** 2
    
    return dydt

We won't implement a function to compute the `Jacobian`, but we will tell `odeint` that the _Jacobian matrix is banded_. This allows the `underlying solver (LSODA)` to _avoid computing values that it knows are zero_. For a _large system_, this `improves the performance significantly`.

First, we define the required inputs.

In [4]:
rng = np.random.default_rng()
y0 = rng.standard_normal(5000)
t = np.linspace(0, 50, 11)
f = 0.024
k = 0.055
du = 0.01
dv = 0.005
dx = 0.025

Time the computation without taking advantage of the banded structure of the Jacobian matrix.

In [5]:
%timeit sol_a = integrate.odeint(grayscott1d, y0, t, args=(f, k, du, dv, dx))

23.6 s ± 2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
start = time()
sol_a = integrate.odeint(grayscott1d, y0, t, args=(f, k, du, dv, dx))
print(f"{sol_a = } was computed in {time() - start} s")

sol_a = array([[-6.31199516e-01,  3.88450717e-01,  9.07134703e-01, ...,
         1.75852130e+00, -5.28519927e-02,  1.12115023e+00],
       [ 3.21700182e-02, -2.07069082e-01,  3.23724205e-02, ...,
        -3.53690019e-02, -1.96014963e-01, -3.47187348e-02],
       [ 1.61260202e-01, -9.40279866e-02,  1.61417579e-01, ...,
        -2.98670551e-02, -3.09202208e-02, -2.99552994e-02],
       ...,
       [ 6.10375415e-01, -3.75615093e-03,  6.10378846e-01, ...,
         1.05475139e-03,  5.23527977e-01,  1.05085428e-03],
       [ 6.54791778e-01, -2.37332321e-03,  6.54793656e-01, ...,
         9.00097642e-04,  5.79264339e-01,  8.98000145e-04],
       [ 6.94031945e-01, -1.51169021e-03,  6.94033034e-01, ...,
         7.09655885e-04,  6.28318772e-01,  7.08507201e-04]]) was computed in 22.93601131439209 s


Now set `ml=2` and `mu=2`, so `odeint` knows that the `Jacobian matrix` is __banded__.

In [7]:
%timeit
sol_b = integrate.odeint(grayscott1d, y0, t, args=(f, k, du, dv, dx), ml=2, mu=2)

In [8]:
start = time()
sol_b = integrate.odeint(grayscott1d, y0, t, args=(f, k, du, dv, dx), ml=2, mu=2)
print(f"{sol_b = } was computed in {time() - start} s")

sol_b = array([[-6.31199516e-01,  3.88450717e-01,  9.07134703e-01, ...,
         1.75852130e+00, -5.28519927e-02,  1.12115023e+00],
       [ 3.21700182e-02, -2.07069082e-01,  3.23724205e-02, ...,
        -3.53690019e-02, -1.96014963e-01, -3.47187348e-02],
       [ 1.61260202e-01, -9.40279866e-02,  1.61417579e-01, ...,
        -2.98670551e-02, -3.09202208e-02, -2.99552994e-02],
       ...,
       [ 6.10375415e-01, -3.75615093e-03,  6.10378846e-01, ...,
         1.05475139e-03,  5.23527977e-01,  1.05085428e-03],
       [ 6.54791778e-01, -2.37332321e-03,  6.54793656e-01, ...,
         9.00097642e-04,  5.79264339e-01,  8.98000145e-04],
       [ 6.94031945e-01, -1.51169021e-03,  6.94033034e-01, ...,
         7.09655885e-04,  6.28318772e-01,  7.08507201e-04]]) was computed in 0.30499863624572754 s


This is really fast. Let's ensure that they have computed the same result.

In [9]:
np.allclose(sol_a, sol_b)

True